kernel : 3.11.5

In [1]:
from collections import Counter
import numpy as np
from PIL import Image
from sklearn.cluster import KMeans

def find_least_dominant_color(image_path, k=8):

    try:
        image = Image.open(image_path)
        
        if image.mode not in ['RGB', 'RGBA']:
            print(f"Error: Unsupported image mode '{image.mode}'. Only RGB and RGBA are supported.")
            return None

        image.thumbnail((150, 150))
        np_image = np.array(image)
        
        pixels = np_image.reshape(-1, np_image.shape[2])
        if image.mode == 'RGBA':
            pixels = pixels[:, :3] 

        kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
        kmeans.fit(pixels)
        
        unique, counts = np.unique(kmeans.labels_, return_counts=True)
        least_common_cluster_index = np.argmin(counts)
        least_dominant_color = tuple(int(c) for c in kmeans.cluster_centers_[least_common_cluster_index])
        
        return least_dominant_color
    except Exception as e:
        print(f"Error finding least dominant color: {e}")
        return None

dominant_color_rgb = find_least_dominant_color('test1.jpg')
print("Dominant color (R, G, B):", tuple(dominant_color_rgb))

Dominant color (R, G, B): (77, 66, 26)


In [2]:
from PIL import Image
import numpy as np
from sklearn.cluster import KMeans
import math

def find_least_dominant_color(image_path, k=8):

    try:
        image = Image.open(image_path)
        
        if image.mode not in ['RGB', 'RGBA']:
            print(f"Error: Unsupported image mode '{image.mode}'. Only RGB and RGBA are supported.")
            return None

        image.thumbnail((150, 150))
        np_image = np.array(image)
        
        pixels = np_image.reshape(-1, np_image.shape[2])
        if image.mode == 'RGBA':
            pixels = pixels[:, :3] 

        kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
        kmeans.fit(pixels)
        
        unique, counts = np.unique(kmeans.labels_, return_counts=True)
        least_common_cluster_index = np.argmin(counts)
        least_dominant_color = tuple(int(c) for c in kmeans.cluster_centers_[least_common_cluster_index])
        
        return least_dominant_color
    except Exception as e:
        print(f"Error finding least dominant color: {e}")
        return None

def calculate_color_distance(color1, color2):
    return np.sqrt(sum([(c1 - c2) ** 2 for c1, c2 in zip(color1[:3], color2[:3])]))          # Calculates the Euclidean distance between two RGB colors.

def encode_with_dynamic_tolerance(image_path, message):

    print("--- STARTING ENCODING ---")
    print("Step 1: Finding the least dominant color...")
    least_dominant_color = find_least_dominant_color(image_path)
    if least_dominant_color is None:
        return None
    print(f"Least dominant color found: {least_dominant_color}")

    try:
        image = Image.open(image_path)
        if image.mode not in ['RGB', 'RGBA']:
             print(f"Error: Unsupported image mode '{image.mode}' for encoding.")
             return None

        binary_message = ''.join(format(ord(char), '08b') for char in message) + '1111111111111110'
        required_pixels = math.ceil(len(binary_message) / 3)
        print(f"Message requires {required_pixels} pixels to hide.")

        print("Step 2: Calculating precise tolerance and boundary count...")
        all_pixels = list(image.getdata())
        
        distances = [calculate_color_distance(tuple(c & 0xFE for c in p[:3]), least_dominant_color) for p in all_pixels]
        distances.sort()
        
        if required_pixels > len(distances):
            print("Error: Message is too long for the number of pixels in the image.")
            return None
            
        calculated_tolerance = distances[required_pixels - 1]
        
        count_less_than_tolerance = len([d for d in distances if d < calculated_tolerance])
        pixels_from_boundary_to_use = required_pixels - count_less_than_tolerance
        
        print(f"Optimal tolerance: {calculated_tolerance}")
        print(f"Pixels needed from boundary: {pixels_from_boundary_to_use}")

        print("Step 3: Hiding the message using the new key system...")
        message_index = 0
        pixels_used = 0
        pixels_on_boundary_used = 0
        
        img_map = image.load()
        for y in range(image.height):
            for x in range(image.width):
                if message_index < len(binary_message):
                    pixel_color = img_map[x, y]
                    
                    pixel_for_distance_calc = tuple(c & 0xFE for c in pixel_color[:3])
                    distance = calculate_color_distance(pixel_for_distance_calc, least_dominant_color)
                    
                    use_pixel = False
                    if distance < calculated_tolerance:
                        use_pixel = True
                    elif distance == calculated_tolerance:
                        if pixels_on_boundary_used < pixels_from_boundary_to_use:
                            use_pixel = True
                            pixels_on_boundary_used += 1

                    if use_pixel:
                        r, g, b = pixel_color[:3]
                        bits_to_hide = binary_message[message_index : message_index + 3]
                        
                        new_r = r & 0xFE | int(bits_to_hide[0]) if len(bits_to_hide) > 0 else r
                        new_g = g & 0xFE | int(bits_to_hide[1]) if len(bits_to_hide) > 1 else g
                        new_b = b & 0xFE | int(bits_to_hide[2]) if len(bits_to_hide) > 2 else b
                        
                        if image.mode == 'RGBA':
                            img_map[x, y] = (new_r, new_g, new_b, pixel_color[3])
                        else:
                            img_map[x, y] = (new_r, new_g, new_b)

                        message_index += len(bits_to_hide)
                        pixels_used += 1
                else:
                    break
            if message_index >= len(binary_message):
                break
        
        print(f"Message hidden within {pixels_used} pixels.")
        output_filename = 'encoded_least_dominant.png'
        image.save(output_filename)
        print(f"Encoded image saved as '{output_filename}'")
        
        return (least_dominant_color, calculated_tolerance, pixels_from_boundary_to_use)

    except Exception as e:
        print(f"An error occurred during encoding: {e}")
        return None





if __name__ == '__main__':
    original_image_file = 'test1.jpg' 
    secret_message = "There is 10 crore hidden under me!!"

    encoding_keys = encode_with_dynamic_tolerance(original_image_file, secret_message)
    
    if encoding_keys:
        least_dom_color_key, tolerance_key, boundary_key = encoding_keys
        
        print("\n--- KEYS GENERATED ---")
        print(f"Key 1 (Color): {least_dom_color_key}")
        print(f"Key 2 (Tolerance): {tolerance_key}")
        print(f"Key 3 (Boundary Count): {boundary_key}")
        
    else:
        print("\nEncoding failed. Please check the error messages.")



--- STARTING ENCODING ---
Step 1: Finding the least dominant color...
Least dominant color found: (77, 66, 26)
Message requires 99 pixels to hide.
Step 2: Calculating precise tolerance and boundary count...
Optimal tolerance: 5.744562646538029
Pixels needed from boundary: 8
Step 3: Hiding the message using the new key system...
Message hidden within 99 pixels.
Encoded image saved as 'encoded_least_dominant.png'

--- KEYS GENERATED ---
Key 1 (Color): (77, 66, 26)
Key 2 (Tolerance): 5.744562646538029
Key 3 (Boundary Count): 8


In [3]:
def decode_with_key(image_path, key_color, key_tolerance, key_boundary_count):

    print("\n--- STARTING DECODING ---")
    print(f"Using Key Color: {key_color}, Tolerance: {key_tolerance}, Boundary Count: {key_boundary_count}")
    
    try:
        image = Image.open(image_path)
        if image.mode not in ['RGB', 'RGBA']:
             print(f"Error: Unsupported image mode '{image.mode}' for decoding.")
             return "Decoding failed due to unsupported image mode."

        binary_message = ""
        delimiter = '1111111111111110'
        pixels_on_boundary_read = 0

        print("Step 1: Scanning for valid pixels using the 3-part key...")
        for y in range(image.height):
            for x in range(image.width):
                pixel_color = image.getpixel((x, y))

                pixel_for_distance_calc = tuple(c & 0xFE for c in pixel_color[:3])
                distance = calculate_color_distance(pixel_for_distance_calc, key_color)
                
                read_pixel = False
                if distance < key_tolerance:
                    read_pixel = True
                elif distance == key_tolerance:
                    if pixels_on_boundary_read < key_boundary_count:
                        read_pixel = True
                        pixels_on_boundary_read += 1
                
                if read_pixel:
                    r, g, b = pixel_color[:3]
                    binary_message += str(r & 1)
                    binary_message += str(g & 1)
                    binary_message += str(b & 1)

                if delimiter in binary_message:
                    break
            if delimiter in binary_message:
                break
        
        delimiter_pos = binary_message.find(delimiter)
        if delimiter_pos == -1:
            return "Decoding failed. Could not find message delimiter. The key may be incorrect or the image is corrupt."
            
        binary_message = binary_message[:delimiter_pos]
        decoded_message = "".join(chr(int(binary_message[i:i+8], 2)) for i in range(0, len(binary_message), 8))
        return decoded_message

    except Exception as e:
        return f"An error occurred during decoding: {e}"



encoded_image_file = 'encoded_least_dominant.png'
revealed_message = decode_with_key(encoded_image_file, least_dom_color_key, tolerance_key, boundary_key)
print(f"The revealed message is: '{revealed_message}'")


--- STARTING DECODING ---
Using Key Color: (77, 66, 26), Tolerance: 5.744562646538029, Boundary Count: 8
Step 1: Scanning for valid pixels using the 3-part key...
The revealed message is: 'There is 10 crore hidden under me!!'


In [4]:
def create_visualization(original_image_path, key_color, key_tolerance, key_boundary_count):

    try:
        image = Image.open(original_image_path)
        if image.mode not in ['RGB', 'RGBA']:
            print("Visualization skipped: Unsupported image mode.")
            return

        viz_image = image.copy()
        img_map = viz_image.load()
        pixels_on_boundary_highlighted = 0
        highlight_color = (255, 0, 0) # Bright red for visibility

        for y in range(viz_image.height):
            for x in range(viz_image.width):
                pixel_color = img_map[x, y]
                
                pixel_for_distance_calc = tuple(c & 0xFE for c in pixel_color[:3])
                distance = calculate_color_distance(pixel_for_distance_calc, key_color)
                
                highlight_pixel = False
                if distance < key_tolerance:
                    highlight_pixel = True
                elif distance == key_tolerance:
                    if pixels_on_boundary_highlighted < key_boundary_count:
                        highlight_pixel = True
                        pixels_on_boundary_highlighted += 1
                
                if highlight_pixel:
                    if viz_image.mode == 'RGBA':
                        img_map[x, y] = (*highlight_color, pixel_color[3])
                    else:
                        img_map[x, y] = highlight_color
        
        output_filename = 'visualization.png'
        viz_image.save(output_filename)
        print(f"Visualization image saved as '{output_filename}'")

    except Exception as e:
        print(f"An error occurred during visualization creation: {e}")

create_visualization(original_image_file, least_dom_color_key, tolerance_key, boundary_key)


Visualization image saved as 'visualization.png'
